<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Estudiante: 
- Vicente Bórquez Z.


### **Link de repositorio de GitHub:** `https://github.com/VBorquezZ/MDS7202-1-23/tree/main/Lab8`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
df_players['labels'] = (~df_players['National_Position'].isnull()).astype(int)
print(df_players['labels'].value_counts())
print('Ratio class 0 : {}'.format(df_players['labels'].value_counts()[0]/len(df_players)))
print('Ratio class 1 : {}'.format(df_players['labels'].value_counts()[1]/len(df_players)))

0    16513
1     1075
Name: labels, dtype: int64
Ratio class 0 : 0.9388787809870366
Ratio class 1 : 0.061121219012963385


Ya se tienen las labels para clasificación hechas. Las clases están muy desbalanceadas, hay ~16.5k ejemplos de clase 0 (jugador no seleccionado) y ~1k de clase 1 (jugador seleccionado), es decir, las proporciones de las clases 0-1 son de 0.939-0.061 aproximadamente.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta :**

In [5]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17588 entries, 0 to 17587
Data columns (total 40 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               17588 non-null  object
 1   Nationality        17588 non-null  object
 2   National_Position  1075 non-null   object
 3   Club_Position      17587 non-null  object
 4   Height             17588 non-null  int64 
 5   Weight             17588 non-null  int64 
 6   Preffered_Foot     17588 non-null  object
 7   Age                17588 non-null  int64 
 8   Work_Rate          17588 non-null  object
 9   Weak_foot          17588 non-null  int64 
 10  Skill_Moves        17588 non-null  int64 
 11  Ball_Control       17588 non-null  int64 
 12  Dribbling          17588 non-null  int64 
 13  Marking            17588 non-null  int64 
 14  Sliding_Tackle     17588 non-null  int64 
 15  Standing_Tackle    17588 non-null  int64 
 16  Aggression         17588 non-null  int64

La columna que tiene más datos nulos es "National_Position", no se usará esta columna para entrenar los clasificadores, pues es justamente lo que se quiere averiguar (para ello se tiene la columna 'labels').
Existe una única excepción a esto y es un jugador que no tiene información en la posición que juega en su club. Es un único dato y este será borrado.
Tampoco se usará el nombre de los jugadores (no aportan información relevante para el problema). 
Tampoco se usará la nacionalidad, pues puede aumentar demasiado la dimensionalidad y por ende la complejidad del problema, no obstante, si no se alcanzan resultados deseados, entonces esto se cambiará.
A todas las demás columnas, si son variables categóricas, se les aplicará OneHot encoding (dada su naturaleza).
Las variables numéricas se escalarán con minmax scaler entre 0 y 1.


In [6]:
df_playersC = df_players.dropna(subset=['Club_Position'])

In [7]:
numeric_cols = ['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 
                'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions', 
                'Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 
                'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 
                'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 
                'Penalties', 'Volleys']
categorical_cols = ['Club_Position', 'Preffered_Foot', 'Work_Rate']

In [9]:
Col_T = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse = False),  categorical_cols),
        ('NumericalTransform', MinMaxScaler(), numeric_cols), 
        ])

In [52]:
df_preprocessed = Col_T.fit_transform(df_playersC)

c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Dada la guía provista, se decide utilizar SVC lineal como parte del primer pipeline. Para los otros dos se usa Kneighbors (asumiendo la probabilidad de que SVC no funcione) y por último, random forest.

In [10]:
# Los pipelines:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

SVCL_pipe = Pipeline([
    ("preprocesamiento", Col_T), 
    ('svc_linear', SVC(kernel='linear'))
])
KN_pipe = Pipeline([
    ("preprocesamiento", Col_T), 
    ('knn', KNeighborsClassifier())
])
RF_pipe = Pipeline([
    ("preprocesamiento", Col_T), 
    ('rf', RandomForestClassifier())
])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [11]:
features = df_playersC.drop(columns=["labels"])
labels = df_playersC.loc[:, "labels"]


X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels, random_state=42
)

print('Balance clases dataset original : ')
print(labels.value_counts() / labels.count() * 100)
print('Balance clases dataset entrenamiento : ')
print(y_train.value_counts() / y_train.count() * 100)
print('Balance clases dataset testeo : ')
print(y_test.value_counts() / y_test.count() * 100)

Balance clases dataset original : 
0    93.887531
1     6.112469
Name: labels, dtype: float64
Balance clases dataset entrenamiento : 
0    93.889502
1     6.110498
Name: labels, dtype: float64
Balance clases dataset testeo : 
0    93.883529
1     6.116471
Name: labels, dtype: float64


La proporción de datos se mantiene estable tras la división de Train/Test.

In [12]:
# Entrenando pipelines
SVCL_pipe = SVCL_pipe.fit(X_train, y_train)
KN_pipe = KN_pipe.fit(X_train, y_train)
RF_pipe = RF_pipe.fit(X_train, y_train)

c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
y_pred_SVCL = SVCL_pipe.predict(X_test)
y_pred_KN = KN_pipe.predict(X_test)
y_pred_RF = RF_pipe.predict(X_test)

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
print(" -------- SVM Lineal -------- ")
print("Matriz de confusión Lineal\n", confusion_matrix(y_test, y_pred_SVCL), "\n")
print(classification_report(y_test, y_pred_SVCL))


 -------- SVM Lineal -------- 
Matriz de confusión Lineal
 [[5449    0]
 [ 355    0]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5449
           1       0.00      0.00      0.00       355

    accuracy                           0.94      5804
   macro avg       0.47      0.50      0.48      5804
weighted avg       0.88      0.94      0.91      5804



c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
print(" -------- KNeighbors -------- ")
print("Matriz de confusión KNeighbors \n", confusion_matrix(y_test, y_pred_KN), "\n")
print(classification_report(y_test, y_pred_KN))

 -------- KNeighbors -------- 
Matriz de confusión KNeighbors 
 [[5408   41]
 [ 339   16]] 

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5449
           1       0.28      0.05      0.08       355

    accuracy                           0.93      5804
   macro avg       0.61      0.52      0.52      5804
weighted avg       0.90      0.93      0.91      5804



In [16]:
print(" -------- RandomForest -------- ")
print("Matriz de confusión RandomForest\n", confusion_matrix(y_test, y_pred_RF), "\n")
print(classification_report(y_test, y_pred_RF))

 -------- RandomForest -------- 
Matriz de confusión RandomForest
 [[5434   15]
 [ 330   25]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5449
           1       0.62      0.07      0.13       355

    accuracy                           0.94      5804
   macro avg       0.78      0.53      0.55      5804
weighted avg       0.92      0.94      0.92      5804



**Comentarios :**

Hay un problema claro por parte de los tres algoritmos, y es el hecho de que tienden a predecir casi siempre que la clase es la 0, basta con mirar los *macro avg f1-scores*: todos son 0.5 +- 0.05. Por lo que son, en general, malos clasificadores.
Aun así, de los 3, el mejor RandomForest: Es el clasificador que obtiene mejores metricas *macro* (considerando el desbalance de clases) y es el que, para la clase 1 obtiene las mejores métricas (tanto precision, como recall y f1).... lo que no quiere decir que que sean muy buenas.

Si existen posibilidades de mejora:
1. Aplicar técnicas de data augmentation sobre los jugadores de clase 1 para balancear las clases (añadiendo variaciones pequeñas a las variables numéricas por ejemplo)
2. Probar nuevos hiperparámetros de los clasificadores (se usaron los que vienen por defecto).
3. Probar otro tipo de transformaciones en el ColumnTransformer.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [17]:
df_playersC.columns

Index(['Name', 'Nationality', 'National_Position', 'Club_Position', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys', 'labels'],
      dtype='object')

In [18]:
def get_position(club_pos):
    ataque = ['ST', 'CF'] 
    central_ataque = ['RW', 'CAM', 'LW'] 
    central = ['RM', 'CM', 'LM'] 
    central_defensa = ['RWB', 'CDM', 'LWB']
    defensa = ['RB', 'CB', 'LB']
    arquero = ['GK']
    
    if club_pos in ataque:
        return 'ataque'
    elif club_pos in central_ataque:
        return 'central_ataque'
    elif club_pos in central:
        return 'central'
    elif club_pos in central_defensa:
        return 'central_defensa'
    elif club_pos in defensa:
        return 'defensa'
    elif club_pos in arquero:
        return 'arquero'
    else:
        return 'Otra'
df_playersC['position'] = df_playersC['Club_Position'].apply(get_position)

C:\Users\borqu\AppData\Local\Temp\ipykernel_22492\2507505969.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playersC['position'] = df_playersC['Club_Position'].apply(get_position)


In [19]:
print(df_playersC['position'].value_counts() / df_playersC['position'].count() * 100)

Otra               77.602775
defensa             6.709501
central             5.157218
arquero             3.593563
central_ataque      3.303577
ataque              2.444988
central_defensa     1.188378
Name: position, dtype: float64


Aproximadamente el 77% de los datos tienen una posición de una clase distinta a las señaladas, estos serán eliminados. 
Con respecto a las clases objetivo (las 5 dadas), se puede ver que, aunque no están perfectamente balanceadas, el desbalance no es tan grande, ya que inclusive entre la clase más repetida (defensa) y la menos repetida (central_defensa), la diferencia no alcanza a ser de un orden de magnitud (es de x6 aproximadamente).


In [20]:
df_position = df_playersC.drop(df_playersC[df_playersC['position'] == 'Otra'].index)

In [21]:
len(df_playersC)-len(df_position)

13648

Se eliminaron 13648 datos

Por clase quedan:

In [22]:
print(df_position['position'].value_counts())

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: position, dtype: int64


In [23]:
# Ahora se crea el nuevo pipeline
# Como se predice la posición, ya no se puede usar 'Club_Position' en las features de entrenamiento/predicción:
categorical_cols_pos = ['Preffered_Foot', 'Work_Rate']
Col_T_pos = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse = False),  categorical_cols_pos),
        ('NumericalTransform', MinMaxScaler(), numeric_cols), 
        ])

In [24]:
features = df_position.drop(columns=["position"])
labels = df_position.loc[:, "position"]


X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels, random_state=42
)

print('Balance clases dataset original : ')
print(labels.value_counts() / labels.count() * 100)
print('Balance clases dataset entrenamiento : ')
print(y_train.value_counts() / y_train.count() * 100)
print('Balance clases dataset testeo : ')
print(y_test.value_counts() / y_test.count() * 100)

Balance clases dataset original : 
defensa            29.956842
central            23.026149
arquero            16.044681
central_ataque     14.749937
ataque             10.916476
central_defensa     5.305915
Name: position, dtype: float64
Balance clases dataset entrenamiento : 
defensa            29.973475
central            23.039030
arquero            16.028799
central_ataque     14.740432
ataque             10.913225
central_defensa     5.305040
Name: position, dtype: float64
Balance clases dataset testeo : 
defensa            29.923077
central            23.000000
arquero            16.076923
central_ataque     14.769231
ataque             10.923077
central_defensa     5.307692
Name: position, dtype: float64


La proporción de datos se mantiene estable tras la división de Train/Test.

Usando la guía provista y por los mismos motivos que en el ítem anterior, la opción de clasificador escogida es SVM lineal.

In [25]:
# Los pipelines:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

# Como las svm son para clasificación binaria, hay que usar OneVsRestClassifier en conjunto con SVC:
SVCL_pos_pipe = Pipeline([
    ("preprocesamiento", Col_T_pos), 
    ('svc_linear', OneVsRestClassifier(SVC(kernel='linear')))
])


In [26]:
# Entrenando pipeline
SVCL_pos_pipe = SVCL_pos_pipe.fit(X_train, y_train)
y_pred_SVCL = SVCL_pos_pipe.predict(X_test)


c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [27]:
y_test.value_counts()

defensa            389
central            299
arquero            209
central_ataque     192
ataque             142
central_defensa     69
Name: position, dtype: int64

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
print(" -------- SVM Lineal -------- ")
print("Matriz de confusión Lineal\n", confusion_matrix(y_test, y_pred_SVCL), "\n")
print(classification_report(y_test, y_pred_SVCL))

 -------- SVM Lineal -------- 
Matriz de confusión Lineal
 [[209   0   0   0   0   0]
 [  0 127  14   0   0   1]
 [  0  19 209  10   6  55]
 [  0  26 142   9   2  13]
 [  0   1  10   3   5  50]
 [  0   2   3   0   3 381]] 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.73      0.89      0.80       142
        central       0.55      0.70      0.62       299
 central_ataque       0.41      0.05      0.08       192
central_defensa       0.31      0.07      0.12        69
        defensa       0.76      0.98      0.86       389

       accuracy                           0.72      1300
      macro avg       0.63      0.62      0.58      1300
   weighted avg       0.67      0.72      0.67      1300



**Comentarios :**

1. No solamente el *f1-score* (*macro avg*) ha mejorado con respecto al problema previo, sino que también hay que considerar que ahora se tienen 5 clases en lugar de dos, por lo que una métrica de 0.58 indica una mejora importante.
2. La clase de pertore tiene métricas perfectas, no hubo confusión alguna entre arqueros y demás jugadores, esto es razonable debido a que los arqueros juegan bajo reglas distintas a los demás jugadores (pueden tocar la pelota con la mano), por lo que hay jugadas que solamente ellos puedn realizar (atajar penales por ejemplo), lo que se ve reflejado en las features de clasificación. En conclusión, determinar si un jugador es o no arquero, usando las features dadas, es un problema trivial.
3. central_defensa es la posición que peores métricas tiene. Este suceso puede deberse a que es justamente la clase menos representada, o que al tener solamente 69 casos de 1300 en el conjunto de testeo, estas métricas no sean representativas.
4. La clase defens es la que tiene la recall más alta (ignorando a los arqueros) de 0.98, pero con una precision de 0.76. Esto útlimo indica que, probablemente el clasificador, ante dudas, indica esta clase. Coincidentemente esta es la clase más representada en el conjunto de entrenamiento.
5. En base a los últimos 2 puntos, se supone que el desbalance de clases sigue siendo un problema (aunqeu menos que en el problema binario).
6. Las mismas ideas para mejorar métricas mencionadas en el ítem previo, aplican en este caso.  

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [29]:
df_sueldos = pd.read_csv('salarios.csv')
df_sueldos

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [30]:
df_playersC

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,labels,position
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,85,92,93,90,81,76,85,88,1,central_ataque
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,71,85,95,88,89,90,74,85,1,central_ataque
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,62,78,89,77,79,84,81,83,1,central_ataque
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,77,87,94,86,86,84,85,88,1,ataque
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,25,25,13,16,14,11,47,11,1,arquero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,14,16,13,13,11,13,15,12,0,Otra
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,12,17,12,11,12,13,16,12,0,Otra
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,41,44,28,42,35,36,42,37,0,Otra
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,15,23,14,12,13,12,24,12,0,Otra


In [33]:
# Concatenamos los dataframes

df_Pstats_Psalary = pd.merge(df_playersC, df_sueldos, left_on='Name', right_on='Player')

df_Pstats_Psalary

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,labels,position,Unnamed: 0,Player,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,81,76,85,88,1,central_ataque,1006,Cristiano Ronaldo,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,88,89,90,74,85,1,central_ataque,1528,Lionel Messi,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,77,79,84,81,83,1,central_ataque,2004,Neymar,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,86,84,85,88,1,ataque,1531,Luis Suárez,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,16,14,11,47,11,1,arquero,498,Manuel Neuer,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,NaN,Res,191,83,Right,18,Medium / Medium,3,...,10,12,8,15,5,0,Otra,973,Phillip Menzel,2034.0
1862,Manuel Akanji,Switzerland,NaN,Sub,187,85,Right,21,Medium / Medium,2,...,18,26,27,40,31,0,Otra,600,Manuel Akanji,54176.0
1863,Moritz Nicolas,Germany,NaN,Res,195,87,Right,19,Medium / Medium,2,...,10,17,14,17,11,0,Otra,965,Moritz Nicolas,2262.0
1864,Giacomo Satalino,Italy,NaN,Sub,188,70,Right,17,Medium / Medium,1,...,5,13,11,18,9,0,Otra,1508,Giacomo Satalino,2827.0


Notar cómo han quedado solamente 1866 casos para los que se conocen tanto las estadísticas de juego como su salario. Dada la naturaleza del problema pedido, son los únicos casos que vale la pena estudiar.

In [99]:
numeric_cols = ['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 
                'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions', 
                'Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 
                'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 
                'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 
                'Penalties', 'Volleys']
categorical_cols = ['Club_Position', 'Preffered_Foot', 'Work_Rate']

# Notar que como se usará un ColumnTransformer, no es necesario hacer el drop de las siguientes columnas. 
# Esto se realiza igualmente para dejar bien claras las columnas que no son relevantes (o apropiadas) para resolver el problema.
features = df_Pstats_Psalary.drop(columns=["Weekly Salary", "Unnamed: 0", "Name", "Player", "position", "labels", "Nationality"])
targets = df_Pstats_Psalary["Weekly Salary"]

X_train, X_test, y_train, y_test = train_test_split(
    features, targets, test_size=0.33, shuffle=True, random_state=42
)

In [152]:
print(len(X_train))
print(len(X_test))

1250
616


In [100]:
from sklearn.preprocessing import StandardScaler
Col_T_salary = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse = False),  categorical_cols),
        ('NumericalTransform', StandardScaler(), numeric_cols), 
        ])

In [102]:
from sklearn import linear_model
from sklearn.metrics import r2_score
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', linear_model.Lasso(alpha=1.0))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))



R² obtenido: 0.0825020722708727


c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [126]:
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', linear_model.Lasso(alpha=10000,tol=1.0))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))



R² obtenido: 0.22340797489757824


c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [151]:
from sklearn import svm
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', svm.SVR(kernel='linear'))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))



R² obtenido: -0.05788933680736297


c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [136]:
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', linear_model.Ridge(alpha=1000))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))



R² obtenido: 0.20292511044015726


c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [142]:
from sklearn.ensemble import AdaBoostRegressor
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', AdaBoostRegressor(learning_rate = 0.01, n_estimators=150, loss = 'exponential'))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))

c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(




R² obtenido: 0.3322958362121796


In [132]:
from sklearn.ensemble import RandomForestRegressor
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', RandomForestRegressor(n_estimators=100))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))

c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(




R² obtenido: 0.4289328910453949


In [145]:
salary_pipe = Pipeline([
    ('preprocesamiento', Col_T_salary),
    ('regresor', AdaBoostRegressor(estimator = RandomForestRegressor(n_estimators=20), 
                                   learning_rate = 0.01, 
                                   n_estimators=100, 
                                   loss = 'exponential'))
])

#Ahora se entrena:
salary_pipe.fit(X_train, y_train)
y_pred = salary_pipe.predict(X_test)

print('\n\nR² obtenido:', r2_score(y_test, y_pred))

c:\Users\borqu\anaconda3\envs\LabPC\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(




R² obtenido: 0.4484117478868468


Siguiendo la guía proporcionada, se probaron varios modelos (con distintos hiperparámetros) de regresión: Lasso, SVR(lineal), Ridge  y Ensemblers (RandomForestRegressor). Esta sucesión de regresores probados se hizo de hecho siguiendo la guía entregada: cuando no se obtuvo un modelo satisfactorio, se probó el siguiente (y esto sucesivamente hasta llegar a los ensamblers).

Los únicos regresores que lograron entregar métrica R² superior a 0.3 fueron los ensamblers: AdaBoostRegressor (su primer uso) obtuvo 0.33 y RandomForestRegressor de 0.42 (métrica objetivo cumplida). Aunque se alcanzó la métrica objetivo con el uso de RandomForestRegressor, para concluir se probó uno combinación de los dos ensamblers que funcionaron: 
* Por defecto, AdaBoostRegressor utiliza como regresor base un DecisionTreeRegressor. Como RandomForestRegressor entregó una mejor métrica, se volvió a probar AdaBoostRegressor pero ahora utilizando RandomForestRegressor como estimador (última celda de código). Con esta modificación se logró mejorar la métrica R² obtenida a un 0.448 (por lo que fue una buena decisión).


**Respuesta**:

1. El coeficiente de determinación R², es una métrica de evaluación de modelos de regresión. Es una medida de la varianza explicada por el modelo, es decir, de los valores predichos, en relación con la varianza total de los datos, es decir, los valores (targets) reales.

2. Para determinar si se pueden obtener mejores resultados es importante notar que se probaron múltiples regresores obteniendo valores para R² en un amplio rango: (-0.057, 0.448). Es claro que los mejores regresores fueron los ensamblers, y aún así entr ellos muestran diferencias importantes: AdaBoostRegressor dependiendo de los hiperparámetros usados marcó una métrica R² de 0.332 o 0.448. Por lo mismo, una primera aproximación para buscar mejores resultados sería probar nuevos hiperparámetros de dicho regresor o inclusive otro tipo de ensambler.

Por supuesto que otra opción importante a considerar sería aumentar la cantidad de datos: se tienen en total 1866 datos, dejando (según la división hecha) solamente 1250 casos para ajustar los regresores. Existiendo muchos más jugadores (que sí existen) se podrían entrenar mejores regresores, inclusive se podrían repetir jugadores pero en distintas épocas de su carrera (por ende distintos stats y distintos sueldos), aumentando aún más la cantidad de datos para entrenar.

No se considera necesario añadir nuevas características a los datos, ya que con las características que ya se contaba y teniendo (relativamente) pocos datos, se logró una métrica aceptable, pues Renacin esperaba como mínimo un regresor que reportara  R² de 0.35 y se entrega uno que reporta 0.448. 

Por todo lo anteriormente dicho, se pronen las dos siguientes estrategias a seguir para conseguir mejores resultados:

1. Probar más ensamblers como regresores y probar distintos hiperparámetros.
2. Aumentar la cantidad de datos.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>